In [2]:
from functools import reduce
import gc
import numpy as np

"""
STVM 계산(3차로 점선) + 지상부 추가 _ 분석시간 증가
    집계시간 : 5분
    분석시간 1800~6900
    검지기 : 280개
"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 6900

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 본선부 검지기
main_line = 60

# 지상부_진출 검지기
exit_line = 265
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도
def density_mean(original_df):
    copy_df = original_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):
    copy_df = original_df.copy()
    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C
    max_capacity = 6600
    entry_saturation_df = (
        # 진입부에서 유입된 교통량이므로 진입부 중 한 개의 검지기를 선정하여 측정
        copy_df[copy_df["New_Measurement"] == enter_line]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0값으로 fix
    main_df["RFR"] = 0
    return main_df

# 유출 정상성 비율
def output_normality(original_df):
    copy_df = original_df.copy()
    # 6번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    entry_df = copy_df[copy_df["New_Measurement"] == enter_line][["VehNo", "t(Entry)"]]

    # 265번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    exit_df = copy_df[copy_df["New_Measurement"] == exit_line][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )
    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"] == enter_line]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"] == exit_line]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # 1−F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
    merged_counts["1-F(outrate)"] = 1 - merged_counts["F(outrate)"]
    return merged_counts


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):


    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
    .merge(entry_saturation_df[["TimeGroup", "Phi_진입"]], on="TimeGroup")
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "1-F(outrate)"]], on="TimeGroup")
    )

    merged_df["StartTime"] = merged_df["TimeGroup"].str.split("~").str[0].astype(int)
    merged_df = merged_df[merged_df["StartTime"] < 5400]
    display("merged_df : ", merged_df)

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["1-F(outrate)"]
    )

    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    # 지상부 1~5, 276~280 제거
    modify_df = df.copy()
    remove_values = list(range(1, 6)) + list(range(276, 281))

    modify_df = modify_df[~modify_df["New_Measurement"].isin(remove_values)]
    modify_df["New_Measurement"] = modify_df["New_Measurement"].apply(
        lambda x: -5 + (x - 6) if 6 <= x <= 10 else (x - 10 if x >= 11 else x)
    )
    return modify_df

def calculate_avg(df):

    def avg_range(df, low, high):
        cols = [c for c in df.columns if low <= c <= high]
        return df[cols].mean(axis=1) if cols else np.nan

    avg_df = pd.DataFrame(index=df.index)
    avg_df["지상부_진입"] = avg_range(df, -5, -1)
    avg_df["진입부"] = avg_range(df, 1, 30)
    avg_df["본선부"] = avg_range(df, 31, 230)
    avg_df["진출부"] = avg_range(df, 231, 260)
    avg_df["지상부_진출"] = avg_range(df, 261, 265)

    return avg_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "1-F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\network_version2.13.0"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)
                display("original_df : ", original_df)

                # 평균속도
                speed_df = speed_mean(original_df)

                # 밀도
                density_df = density_mean(speed_df)

                # 가중 중차량 비율
                heavy_df = heavy_rate(original_df)

                # 진입 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 유출입 비율
                rfr_df = rfr_rate(original_df)

                # 진출 정상성
                normality_df = output_normality(original_df)
                display("normality_df : ", normality_df )
                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)


                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                display("z_score_df : ", z_score_df)
                save_to_excel(z_score_df, folder_path, "환산점수", i)

                # 변수별 TimeGroup별 평균
                variable_time_df = variable_timegroup_avg(stvm_df)
                #save_to_excel(variable_time_df, folder_path, "변수별 평균", i)

                # 변수별 전체 평균
                variable_total_df = variable_total_avg(variable_time_df)
                #save_to_excel(variable_total_df, folder_path, "변수별 총 평균", i)

                # STVM 피봇
                stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도, 밀도 전체 평균값
                speed_density_df = speed_density_avg(density_df)
                #save_to_excel(speed_density_df, folder_path, "속도_밀도 평균값", i)

                # 속도 변화 피봇
                delta_speed_pivot_df = pivot_table(speed_df, "delta_V", preprocess=modify_frame)
                #save_to_excel(delta_speed_pivot_df, folder_path, "속도변화량", i)

                # 밀도 변화 피봇
                delta_density_pivot_df = pivot_table(density_df, "delta_K", preprocess=modify_frame)
                #save_to_excel(delta_density_pivot_df, folder_path, "밀도변화량", i)

                # 속도값 피봇
                speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 가중평균속도
                weighted_speed_df = weighted_avg_speed(original_df)
                weighted_speed_df = pivot_table(weighted_speed_df, "Weighted_Avg_Speed", preprocess=modify_frame)
                #save_to_excel(weighted_speed_df, folder_path, "가중평균속도", i)
                # 메모리 정리
                del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()

'original_df : '

,Measurem.,t(Entry),t(Exit),VehNo,Vehicle type,Line,v[km/h],VehLength[m],,New_Measurement,TimeGroup
0,10010,1800.08,-1.0,2489,100,0,98.2,4.61,NaN,10,1800~2100
1,10020,1800.01,-1.0,2456,100,0,106.5,4.21,NaN,20,1800~2100
2,10022,1800.01,-1.0,2422,640,0,86.4,10.21,NaN,22,1800~2100
3,10052,1800.05,-1.0,2290,100,0,102.9,4.21,NaN,52,1800~2100
4,10053,1800.10,-1.0,2273,630,0,99.2,6.67,NaN,53,1800~2100
...,...,...,...,...,...,...,...,...,...,...,...
2168446,30200,6899.98,-1.0,9030,630,0,45.0,6.67,NaN,200,6600~6900
2168447,30224,6899.98,-1.0,8593,100,0,30.2,4.64,NaN,224,6600~6900
2168448,30232,6899.98,-1.0,8506,100,0,16.5,4.76,NaN,232,6600~6900
2168449,30233,6899.99,-1.0,8375,100,0,38.2,4.01,NaN,233,6600~6900


'normality_df : '

,TimeGroup,Q_in,Q_out,Q_out_shift,F(outrate),1-F(outrate)
0,1800~2100,390,378,450.0,1.153846,-0.153846
1,2100~2400,425,389,358.0,0.842353,0.157647
2,2400~2700,420,422,435.0,1.035714,-0.035714
3,2700~3000,397,436,393.0,0.989924,0.010076
4,3000~3300,525,450,353.0,0.672381,0.327619
5,3300~3600,534,358,409.0,0.765918,0.234082
6,3600~3900,534,435,435.0,0.814607,0.185393
7,3900~4200,541,393,416.0,0.768946,0.231054
8,4200~4500,533,353,431.0,0.808630,0.191370
9,4500~4800,536,409,412.0,0.768657,0.231343


'merged_df : '

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime
0,1800~2100,1,0.006673,0.000897,0.303030,0.709091,0,-0.153846,1800
1,1800~2100,2,0.003166,-0.010651,0.305764,0.709091,0,-0.153846,1800
2,1800~2100,3,-0.005473,0.008042,0.310606,0.709091,0,-0.153846,1800
3,1800~2100,4,-0.002535,0.000016,0.309824,0.709091,0,-0.153846,1800
4,1800~2100,5,0.000663,-0.015804,0.310606,0.709091,0,-0.153846,1800
...,...,...,...,...,...,...,...,...,...
3355,5100~5400,276,0.007865,0.001806,0.515738,0.992727,0,0.168498,5100
3356,5100~5400,277,0.002100,-0.002096,0.510791,0.992727,0,0.168498,5100
3357,5100~5400,278,0.000645,-0.012627,0.508393,0.992727,0,0.168498,5100
3358,5100~5400,279,0.025909,-0.041816,0.509709,0.992727,0,0.168498,5100


'z_score_df : '

New_Measurement,-5,-4,-3,-2,-1,1,2,3,4,5,...,256,257,258,259,260,261,262,263,264,265
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,52.666746,52.691156,52.692958,52.725307,52.676724,52.670965,52.710804,52.717062,52.682802,52.724509,...,52.670576,52.638540,52.706487,52.733261,52.500568,52.660227,52.689405,52.717573,52.749851,52.725840
2100~2400,62.071165,61.821709,62.051767,62.537547,61.596041,61.273016,62.293273,61.631890,61.306909,62.368376,...,62.881550,61.798572,62.490154,62.723064,69.071251,63.894801,62.986495,62.749186,62.856703,63.310619
2400~2700,54.164294,54.693481,54.585065,54.512307,54.789665,54.480884,54.482104,54.917231,54.609626,54.321902,...,53.848020,53.946984,53.782960,54.021845,57.123799,54.734918,54.427517,54.170579,54.319425,54.040631
2700~3000,54.950635,54.837728,54.363902,54.491188,54.370721,54.714964,54.713831,54.349270,54.734990,54.776203,...,54.567558,54.472891,54.404373,54.519919,57.975296,54.707960,54.064863,53.874042,53.948217,54.140248
3000~3300,93.380927,93.175614,93.981213,93.743836,93.609149,93.177306,93.011052,93.408594,93.272739,93.272284,...,82.895135,82.838280,83.209423,83.252813,89.558653,84.608486,84.457111,84.686120,84.356459,83.942426
3300~3600,89.453709,89.497065,89.124449,89.124231,89.075472,89.539717,89.402499,89.036616,88.898356,88.896747,...,81.611911,80.488424,80.851056,81.804920,87.562436,82.550381,81.095368,80.725487,72.438401,80.861331
3600~3900,88.519305,88.451203,88.498299,88.074837,88.072144,88.071462,87.978447,87.693600,87.740109,87.738063,...,71.638869,71.635663,71.838642,71.463473,83.812831,72.066321,71.734941,71.605396,71.719313,71.579463
3900~4200,87.778427,87.894231,87.959871,88.411240,88.798505,88.668913,88.353536,88.302643,88.815395,88.357315,...,86.751394,86.881813,84.560873,84.477452,89.550464,84.660902,84.062208,82.961895,82.549156,82.777884
4200~4500,87.631645,87.268720,87.483477,86.862335,87.169637,87.163481,87.712737,87.999603,87.484836,87.374166,...,86.703624,88.176175,87.754571,88.060247,93.387788,90.489700,88.709867,88.926839,90.055948,89.154997


환산점수_1.xlsx 생성 완료
